### Required packages

In [ ]:
import pickle
import pandas as pd
import numpy as np
import sklearn.metrics as mt
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve

## Load data

In [ ]:
# Most of this cell has been removed, as it contained data with sensitive patient information

columns = ["Geslacht", "Leeftijd", "HB", "ERY", "MCV", "MCH", "TRMB", "LEU", "CRP", "FER", "VITB12", "FOL"]
replace_dict = {"Geslacht": {'M': 1, "V": 0}}

In [ ]:
# Some initial data visualization

plt.scatter(man_df['HB'], man_df['VITB12'])
plt.title("VITB12")
plt.show()

plt.scatter(man_df['HB'], man_df['FER'])
plt.title("FER")
plt.show()

plt.scatter(man_df['HB'], man_df['FOL'])
plt.title("FOL")
plt.show()

In [ ]:
print(man_nofol_df[:-3].isnull().values.any())
print(man_df.dtypes)

## Clean up data, streamline column names and check for null values

In [ ]:
df_man = pd.concat([man_df[columns], man_df_2019[columns], man_df_2020[columns], 
                    man_df_2019_2[columns], man_nofol_df[columns]])
df_woman = pd.concat([woman_df[columns], woman_df_2019[columns], woman_df_2020[columns], woman_nofol_df[columns],
                      woman_df_2019_2[columns]])

columns = ["Geslacht", "Leeftijd", "HB", "ERY", "MCV", "MCH", "TRMB", "LEU", "CRP", "FER"]
for df in [df_man, df_vrouw]:
    df.replace(replace_dict, inplace=True)

    df['Leeftijd'] = df['Leeftijd'].astype(np.float64)
    df['MCV']= df['MCV'].astype(np.float64)
    df['TRMB'] = df['TRMB'].astype(np.float64)
    df['CRP'] = df['CRP'].astype(np.float64)

    # no NANs
    for c in columns:
        print(c)
        print(df[c].isnull().values.any())



## Split up in train and test sets

In [ ]:
# Chec

print(len(df_man) - np.sum(df_man['VITB12'].notna()))
print(len(df_man) - np.sum(df_man['FOL'].notna()))
print(len(df_man) - np.sum(df_man['FER'].notna()))

print(len(df_vrouw) - np.sum(df_vrouw['VITB12'].notna()))
print(len(df_vrouw) - np.sum(df_vrouw['FOL'].notna()))
print(len(df_man) - np.sum(df_man['FER'].notna()))

In [ ]:
predict_name = "FER"
Xnp_man = df_man
ynp_man = pd.cut(df_man["FER"], bins=[-float("inf"),30,float("inf")],labels=[1,0]).to_numpy()
# ynp = np.log10(df["FER"].to_numpy())

# ynp = pd.cut(df["VITB12"], bins=[-float("inf"),200,float("inf")],labels=[0,1]).to_numpy()
# ynp = pd.cut(df["FOL"], bins=[-float("inf"),6,39,float("inf")],labels=[0,1,0], ordered=False).to_numpy()
Xnp_vrouw = df_vrouw
ynp_vrouw = pd.cut(df_vrouw["FER"], bins=[-float("inf"),13,float("inf")],labels=[1,0]).to_numpy()

Xnp = pd.concat([Xnp_man,Xnp_vrouw])
ynp = np.concatenate([ynp_man, ynp_vrouw])

X_train, X_test, ynp_train, ynp_test = train_test_split(Xnp, ynp, test_size=0.3, random_state=42)
Xnp_train = X_train[columns[0:-3]].to_numpy()
Xnp_test = X_test[columns[0:-3]].to_numpy()
ynpCopy_train = ynp_train.copy()
XnpCopy_train = Xnp_train.copy()
print(Xnp_man.shape)
print(Xnp_vrouw.shape)
print(Xnp_train.shape)
print(ynp_train.shape)
print(Xnp_test.shape)
print(ynp_test.shape)
print(len(Xnp))

## Train random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

regr = RandomForestClassifier(random_state=40, class_weight="balanced_subsample")
regr.fit(Xnp_train, ynp_train)
importances = regr.feature_importances_
    

In [ ]:
# Save model

filename = 'random_forest_model_ferritine.sav'
pickle.dump(regr, open(filename, 'wb'))
clf = pickle.load(open(filename, 'rb'))

In [ ]:
# Plot feature importances

importances = regr.feature_importances_
std = np.std([tree.feature_importances_ for tree in regr.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
print(indices)
# Print the feature ranking
print("Feature ranking:")

for ind, f in enumerate(indices):
    print("{}. feature {} ({})".format(ind+1, columns[:-1][f], importances[indices[ind]]))

# Plot the impurity-based feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(Xnp_train.shape[1]), importances[indices],
        color="r", yerr=std[indices], align="center")
plt.xticks(range(Xnp_train.shape[1]), np.asarray(columns[:])[indices])
plt.xlim([-1, Xnp_train.shape[1]])
plt.show()

## Get predictions on test data

In [ ]:
# Check a few output

for i in range(10):
    print("predicted: {}".format(regr.predict(Xnp_test[i].reshape(1, -1))))
    print("proba: {}".format(regr.predict_proba(Xnp_test[i].reshape(1, -1))))
    print("actual: {}".format(ynp_test[i]))


y_pred = regr.predict(Xnp_test)
y_proba = regr.predict_proba(Xnp_test)
print("score: {}".format(regr.score(Xnp_test, ynp_test)))

## Get ROC curve

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt

p_test_pos = y_proba[:,1]
y_roc = np.copy(ynp_test)

print("Area under ROC curve ", mt.roc_auc_score(y_score=p_test_pos, y_true=y_roc))
fpr, tpr, tresh = mt.roc_curve(y_roc, p_test_pos, pos_label=1)
plt.title("ROC predict low {}".format(predict_name))
plt.xlim(-0.005,1.0)
plt.ylim(0.0,1.005)
plt.plot(fpr, tpr)
# plt.savefig("FER_Roc_nofol.png", format="png")

# disp = plot_precision_recall_curve(regr, Xnp_test, ynp_test)
# disp.ax_.set_title('2-class Precision-Recall curve: '
#                    'AP={0:0.2f}'.format(average_precision))

In [ ]:
# FER: Check optimal cutoffs for ferritin levels

predict_name = "FER"
men_cutoff = [18,19,20,21,22,23,24,25,26]
women_cutoff = [6, 7, 8, 9, 10, 11, 12, 13, 14]
for man_cutoff in men_cutoff:
    for woman_cutoff in women_cutoff:
        Xnp_man = df_man
        ynp_man = pd.cut(df_man["FER"], bins=[-float("inf"),man_cutoff,float("inf")],labels=[1,0]).to_numpy()
        Xnp_vrouw = df_vrouw
        ynp_vrouw = pd.cut(df_vrouw["FER"], bins=[-float("inf"),woman_cutoff,float("inf")],labels=[1,0]).to_numpy()
        Xnp = pd.concat([Xnp_man,Xnp_vrouw])
        ynp = np.concatenate([ynp_man, ynp_vrouw])
        X_train, X_test, ynp_train, ynp_test = train_test_split(Xnp, ynp, test_size=0.2, random_state=42)
        Xnp_train = X_train[columns[0:-3]].to_numpy()
        Xnp_test = X_test[columns[0:-3]].to_numpy()
        
        regr = RandomForestClassifier(random_state=42, class_weight="balanced_subsample")
        regr.fit(Xnp_train, ynp_train)
        y_proba = regr.predict_proba(Xnp_test)
        
        p_test_pos = y_proba[:,1]
        # p_test_pos = (y_proba + 1) / 2 #ridge
        y_roc = np.copy(ynp_test)
        print("{} {} {}".format(woman_cutoff, man_cutoff, mt.roc_auc_score(y_score=p_test_pos, y_true=y_roc)))
#         print("women: {}, men: {}, AUC: {}".format(woman_cutoff, man_cutoff, mt.roc_auc_score(y_score=p_test_pos, y_true=y_roc)))

        

In [ ]:
# In this cell we can check our model on independent validation data and write output to excel sheet

X_test['FER']
X_test['model_output'] = y_proba[:,1]
X_test.replace({"Geslacht": {1: 'M', 0: "V"}}, inplace=True)
with pd.ExcelWriter(r"REMOVED") as writer:  
    X_test.to_excel(writer, sheet_name='Validation_data', index=False)